https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

SOS_TOKEN = '<SOS>' # Start Of Word

# Data setup

In [3]:
# importing keras might cause problems with cudann version etc
# import keras # some good utils in here
# path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [4]:
path = '/home/ohu/.keras/datasets/nietzsche.txt'
text = open(path).read().lower()
path
print('corpus length:', len(text))

'/home/ohu/.keras/datasets/nietzsche.txt'

corpus length: 600893


In [5]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [6]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

# Utils

In [8]:
def string_to_tensor(in_str, chars_index=char_indices, as_variable=True):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = torch.zeros(1, tensor_length, len(chars_index))
    for li, letter in enumerate(in_str):
        tensor[0, li, chars_index[letter]] = 1
    if as_variable:
        tensor = Variable(tensor).cuda()
    return tensor

string_to_tensor('hello', as_variable=False).size()

torch.Size([1, 5, 57])

In [9]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Char Rnn

In [23]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CharRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.layers = 1
        
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.lin_output = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_char_vs, hidden = None):
        if hidden is None:
            hidden = self.init_hidden()
        
        all_outputs, hidden = self.rnn(input_char_vs, hidden)
        
        output = self.lin_output(hidden)
        output = F.log_softmax(output[0])
        
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.layers, 1, self.hidden_size)).cuda()
    
model = CharRNN(len(chars), 512).cuda()
model

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

In [12]:
#tmp = Variable(torch.FloatTensor([2]).view(1,1,-1)).cuda()
#tmp = Variable(string_to_tensor('hello')).cuda()
tmp = model(string_to_tensor('hello'))
tmp[0].size()
tmp[1].size()
chars[tmp[0].topk(1)[1].data[0][0]]

torch.Size([1, 57])

torch.Size([1, 1, 512])

'('

# Examples

In [13]:
def print_example(iters=320, choice=True):
    seed_string="ethics is a basic foundation of all that"
    for i in range(iters):
        #x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        x = string_to_tensor(seed_string)
        output, hidden = model(x)
        if choice:
            next_char = np.random.choice(chars, p=F.softmax(output)[0].data.cpu().numpy())
        else:
            next_char_idx = output.topk(1)[1].data[0][0]
            next_char = chars[next_char_idx]
        # return next_char
        seed_string = seed_string + next_char
    print(seed_string)
#tmp = print_example(choice=False)
tmp = print_example(iters=40, choice=False)
tmp

ethics is a basic foundation of all thatävvvv(((((((((((((((((((((((((((((((((((


In [15]:
def get_random_sample(length=40):
    sample_place = random.randint(0, len(text)-1000)
    sample = text[sample_place:sample_place+length]
    return sample
get_random_sample()

' artists ("ah! this dreadful science!" s'

In [77]:
clip_parameters_value = 0.25

def train_single(sample, optimizer, loss_function, use_teacher_forcing):
    model.zero_grad()
    loss = 0
    
    target_line_tensor = Variable(torch.LongTensor([char_indices[s] for s in sample])).cuda()
    output_line = []
    
    input_char = string_to_tensor(sample[0])[0,0]
    
    hidden = model.init_hidden()
    for i in range(len(sample)-1):
        output, hidden = model(input_char.view(1,1,-1), hidden)
        loss += loss_function(output, target_line_tensor[i+1])

        char_idx = output.topk(1)[1].data[0][0]
        if i >= 1000:
            #return input_char
            x = chars[input_char.topk(1)[1].data[0]]
            y = chars[target_line_tensor[i+1].data[0]]
            print(x,y)
        char = chars[char_idx]
        output_line.append(char)
        
        if use_teacher_forcing:
            input_char = sample[i+1] # replace input with right target
        else:
            input_char = char
            
        input_char = string_to_tensor(input_char)[0,0]

    if clip_parameters_value:
        torch.nn.utils.clip_grad_norm(model.parameters(), clip_parameters_value)
    
    loss.backward()
    optimizer.step()

    return ''.join(output_line), loss.data[0] / len(sample)

tmp = get_random_sample(20)
print(tmp)
train_single(tmp, torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=False)
train_single(tmp, torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=True)
#tmp = train_single(tmp, torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=True)

#tmp.topk(1)[1].data[0][0]

od or as bad as
the 


('éyyyyyyyyyyyyyyyyyy', 3.8425880432128907)

('éyuyyuyyuyyyuyyyyyy', 3.830978012084961)

In [78]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000, sample_len=5):
    global optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    start = time.time()
    model.train()

    current_loss = 0
    current_loss_iter = 0
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        s_string = get_random_sample(sample_len)
        
        result, loss = train_single(sample=s_string, optimizer=optimizer, loss_function=nn.NLLLoss(),
                                    use_teacher_forcing=use_teacher_forcing)
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            correct = '✓' if result == s_string else "✗: {}".format(s_string)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} ({}) {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss,
                      result, correct, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        #if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
        #    model_training.save_models()
        #    acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
        #    model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [79]:
from pytorch_utils_oh_3 import ModelTraining
MODEL_SAVE_PATH = 'char_rnn_fast_ai_testing_2'

In [80]:
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

Save path: data/models/char_rnn_fast_ai_testing_2


In [81]:
train_iterations(n_iters=10, print_every=5)

     5  50% (   0m 0s)   3.210   |   3.20: y  r (✗: that,) (forcing)
    10 100% (   0m 0s)   3.202   |   3.15:  eee (✗: , but) 


In [82]:
train_iterations(n_iters=1000, print_every=100, sample_len=40)

   110  10% (   0m 5s)   3.059   |   2.89:                                         (✗: d; prayers for miracles; sins against a ) (forcing)
   210  20% (  0m 11s)   3.003   |   2.74:     s   e         se       e      e     (✗: um? is there not time
enough for that? h) (forcing)
   310  30% (  0m 17s)   2.959   |   2.41: nh an  ae  e  n een  an  n  an  ne  e   (✗: it, and hence it lies in all innocence.
) (forcing)
   410  40% (  0m 23s)   2.928   |   2.88:  an                                     (✗: y such natures may resist the general ex) 
   510  50% (  0m 29s)   2.894   |   2.87: n                                       (✗: and ultimate purpose of all dogmatic eff) 
   610  60% (  0m 35s)   2.861   |   2.30: h inen sg ng he  ee   d  erge e nn  n i (✗: ts origin in the present conception of m) (forcing)
   710  70% (  0m 41s)   2.848   |   2.56: e re    n tn  iihe   en  on e   iner  i (✗: rgument against the thing
agreed upon, f) (forcing)
   810  80% (  0m 47s)   2.836   |   2.56:  e

In [83]:
train_iterations(n_iters=3000, print_every=1000, sample_len=40)

  2010  33% (   1m 1s)   2.627   |   2.86: the   e                                 (✗:  whole course,
anything to serve him as ) 
  3010  67% (   2m 2s)   2.578   |   2.45:  teonnte   tn tfe ngti ootg rle   af te (✗: y science," in ordinary language), on he) (forcing)
  4010 100% (   3m 2s)   2.513   |   1.91: rttettonseeteonson thrpant tle thet tnd (✗: us no conclusion is permissible that any) (forcing)


In [84]:
train_iterations(n_iters=5000, print_every=1000, sample_len=40)

  5010  20% (  0m 59s)   2.514   |   2.97: e for the reas                          (✗: conduct assumes in his eyes
the characte) 
  6010  40% (  1m 58s)   2.455   |   2.13: herso r  

1  tn iontethoe ia ees  ihe  (✗: the soul.

33. it cannot be helped: the ) (forcing)
  7010  60% (  2m 58s)   2.447   |   2.20: e ire tonset oene   in er aemteett ol a (✗: refore cannot give." after so graceful a) (forcing)
  8010  80% (  3m 57s)   2.412   |   3.12: to                                      (✗:  rotundly like
an old, green, heavily-ho) 
  9010 100% (  4m 56s)   2.398   |   3.12:  the sti                                (✗: , nothing but a blank
sameness: how late) 


In [85]:
print_example()

ethics is a basic foundation of all thatiarly pricipilous allion a peysompos a no, these urorvents, the " ampory speci ta tht utoregfeci e of ntittlicias
of which the hourts buthing, the meant of he way
it itself, howes it the posed is it
is excaping, hearseven the shillasion) a may will of bele not one wnowhing coupraizes aid socersely isselves isted the sp


In [87]:
print_example(choice=False)

ethics is a basic foundation of all that it is the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species of the species o


In [ ]:
train_iterations(n_iters=100000, print_every=10000, sample_len=40)

In [ ]:
train_iterations(n_iters=100000, print_every=10000, sample_len=40, teacher_forcing_ratio=0.3)

In [ ]:
train_iterations(n_iters=100000, print_every=10000, sample_len=40, teacher_forcing_ratio=0.2, lr=0.0001)

In [ ]:
train_iterations(n_iters=400000, print_every=10000, sample_len=40, teacher_forcing_ratio=0, lr=0.0001)